### Этот ноутбук содержит процесс создания программы для семантического поиска мемов по текстовому запросу пользователя.


#### Как работает программа?
- **Пользователь пишет короткое текстовое описание ситуации**, к которой он хочет подобрать мемы.
- Запрос передается в **языковую модель-трансформер**, которая превращает запрос в эмбеддинг предложения.
- Далее идет поиск наиболее подходящего мема из датасета с помощью библиотеки **faiss**, основанный на сравнении эмбеддинга запроса и эмбеддингов текстовых описаний мемов на основе **косинусного расстояния**.

#### Какие данные используются?
- Собранные данные с сайта **memepedia.ru** - архив русских мемов.
- Данные собраны с помощью библиотеки **Selenium** и представляют собой csv-файл с названиями мемов, их текстовым описанием и ссылкой на страницу.
- В датасете примерно **2000 мемов**.

#### Используемые библиотеки
- **pandas, numpy, huggingface, sentence_transformers, faiss, torch**

#### План проекта
1. **Чистка данных** после сбора - удаление пустых строк и дубликатов.
2. Поиск наиболее подходящей **языковой модели**. Использование готовых моделей-трансформеров доступных на huggingface сначала без дополнительного обучения.
3. Настройка поиска **faiss**.
4. **Дообучение подходящей модели** на собственных данных. Использование ChatGPT для искусственного дополнения датасета примерами запросов пользователей.
5. **Деплоймент программы** в виде телеграм-бота (не описан в этом ноутбуке).


#### Ссылки на ресурсы
- [Курс по NLP для работы с трансформерами и faiss](https://huggingface.co/learn/nlp-course/chapter5/6?fw=tf)
- [Semantic Search with S-BERT is all you need](https://medium.com/mlearning-ai/semantic-search-with-s-bert-is-all-you-need-951bc710e160)
- [From zero to semantic search embedding model](https://blog.metarank.ai/from-zero-to-semantic-search-embedding-model-592e16d94b61)
- [Train and Fine-Tune Sentence Transformers Models](https://huggingface.co/blog/how-to-train-sentence-transformers)
- [How to easy preprocess Russian text](https://www.kaggle.com/code/alxmamaev/how-to-easy-preprocess-russian-text)
- [The Art of Pooling Embeddings](https://blog.ml6.eu/the-art-of-pooling-embeddings-c56575114cf8)


## 1. Загрузка данных и чистка текста

In [1]:
import pandas as pd 
import numpy as np
from datasets import Dataset


Я собираюсь использовать модели-трансформеры, поэтому единственный этап чистки данных здесь - удаление дубликатов, пустых строк или тех, что попали в датасет случайно при парсинге. Однако я оставила функции, которые изначально использовала для чистки.
- Удаление пунктуации, приведение в нижний регистр и словарную форму не требуется, т.к. трансформеры учитывают общий контекст слов в предложении и в "сыром" формате без обработки может содержаться важная информация. Я пробовала чистить данные ради эксперимента, но это не улучшило качество поиска.
- Также пробовала удалять самые частно встречающиеся слова (мем, тренд, это и т.д.), это тоже не повлияло на результат.
- Единственный вопрос, который остается открытым - исправление орфографических ошибок в сообщении пользователя, т.к. они могут повлиять на результат поиска

In [2]:
meme_data = pd.read_csv('meme_dataset.csv', index_col=0)

# чистим текстовые данные
meme_data = meme_data.drop(index=meme_data[meme_data['meaning'] == 'Галерея'].index)
meme_data = meme_data.drop(index=meme_data[meme_data['meaning']=='\xa0'].index)
meme_data = meme_data.dropna()
meme_data

,page,header,link,meaning
0,1,А мне X дороже Y,https://memepedia.ru/a-mne-x-dorozhe-y/,Мем “А мне X дороже Y” (или “А мне X важнее Y”...
1,1,Рыба с большой головой смотрит на дайвера,https://memepedia.ru/ryba-s-bolshoj-golovoj-sm...,"В мемах, где рыба с большой головой смотрит на..."
2,1,Две девушки в черных платьях,https://memepedia.ru/dve-devushki-v-chernyx-pl...,Мем “Две девушки в черных платьях” показывает ...
3,1,Пивной ежик,https://memepedia.ru/pivnoj-ezhik/,"Мем “Пивной ежик” – это забавный тренд, которы..."
4,1,Война древних русов с ящерами,https://memepedia.ru/vojna-drevnix-rusov-s-yas...,Война древних русов с ящерами или битва русов ...
...,...,...,...,...
3079,133,Мальчик у доски,https://memepedia.ru/malchik-u-doski/,Картинку с мальчиком у доски и словом “Тут” из...
3085,134,Не расстраивай Леонида Аркадьевича,https://memepedia.ru/ne-rasstraivaj-leonida-ar...,Мем “Не расстраивай Леонида Аркадьевича” испол...
3086,134,Девочка с хвостиками,https://memepedia.ru/mem-devochka-s-xvostikami/,Мем с девочкой с хвостиками используется для в...
3087,134,Танец Медведева,https://memepedia.ru/tanec-medvedeva/,"Шутки про танец Медведева обычно используют, ч..."


In [264]:
#убираем пунктуацию и ссылки

import re, string

def remove_symbols(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower().replace('– ', '').replace('“', '').replace('”', '').replace(u'\xa0', u' ')
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text


In [337]:
#лемматизируем текст (приведем слова в словарную форму)

from pymystem3 import Mystem
mystem = Mystem()

def lemmatize_text(x):
    text = mystem.lemmatize(x)
    return ('').join(text)


In [338]:
#удаляем стоп-слова

import nltk
from nltk.corpus import stopwords

stop_words = stopwords.words('russian')

#оставим местоимения
russian_pronouns = 'я, мы, ты, вы, он, она, оно, они, себя, мой, твой, ваш, наш, свой, его, ее, их, то, это, тот, этот, такой, таков, столько, весь, всякий, сам, самый, каждый, любой, иной, другой, кто, что, какой, каков, чей, сколько, никто, ничто, некого, нечего, никакой, ничей, нисколько,кто-то, кое-кто, кто-нибудь, кто-либо, что-то, кое-что, что-нибудь, что-либо, какой-то, какой-либо, какой-нибудь, некто, нечто, некоторый, некий'
russian_pronouns = russian_pronouns.split(', ')

for i_elem in stop_words:
    if i_elem in russian_pronouns:
        stop_words.remove(i_elem)

def remove_stopword(x):
    filtered_text = []
    
    for i_word in x.split():
        if i_word not in stop_words:
            filtered_text.append(i_word)
            
    return (' ').join(filtered_text)
                                

In [10]:
#посмотрим самые часто встречающиеся слова и уберем их
import collections

meme_data['meaning_list'] = meme_data['meaning'].apply(lambda x:x.split())
top = collections.Counter([item for sublist in meme_data['meaning_list'] for item in sublist])
temp = pd.DataFrame(top.most_common(100))
temp.columns = ['Common_words','count']
temp.style.background_gradient(cmap='Blues')

,Common_words,count
0,мема,2013
1,который,1003
2,человек,818
3,это,791
4,становиться,463
5,ситуация,445
6,использоваться,430
7,мочь,428
8,часто,364
9,показывать,331


In [285]:
#добавим наиболее встречающиеся слова, которые не несут смысловой нагрузки (обновим список стоп-слов)

#temp.Common_words.values

new_stopwords = ['мема', 'который', 'ситуация',
       'использоваться', 'часто', 'показывать', 'чтото',
       'например', 'картинка',
       'использовать', 'мем', 'шутка', 'также', 'смысл',
       'высмеивать',
       'обычно', '—',
       'просто', 'делать', 'игра', 'любой', 'очень', 'выражать', 'означать',
       'правило', 'случай', 'иллюстрировать', 'образ', 
       'различный', 'являться', 'качество', 'ироничный', 'иметь', 'время',
       'изза', 'поэтому', 
       'значение', 'тема', 'ктото', 'формат',
       'выражение', 
       'оригинальный', 'кадр', 'явление', 'начинать', 'символизировать',
       'обыгрывать',
       'описывать', 'выглядеть', 'типичный', 'пример', 'тренд']

for i_word in new_stopwords:
    stop_words.append(i_word)


#### Создадим функцию, которая объединяет все шаги чистки данных

In [1]:
def clean_text(text):
    text = remove_symbols(text)
    text = lemmatize_text(text)
    text = remove_stopword(text)
    return text

In [14]:
# example
clean_text(meme_data['meaning'][0])

'x дорогой y x важный y постироничный делаться намеренно кривой шрифт обрезать текст персонаж отметать основной предпочтение говорить любовь чемуто стыдный смешной'

В качестве эксперимента также решила объединить столбцы со значением мема и его заголовком, т.к. чаще всего в заголовке уже содержится какая-то важная информация о значении мема. Но это также особо не улучшило результаты поиска. Скорее всего из-за того, что в описаниях мемов чаще всего уже упоминается его название.

In [309]:
# создадим ф-ю которая добавляет заголовок в столбец со значением 
def add_header(df):
    
    header = df.header
    meaning = df.meaning
    new_text = ''
    
    for i_word in header.split():
        if i_word not in meaning:
            new_text = new_text + i_word + ' '
    
    new_text += meaning
    return new_text

#почистим все данные
meme_data['header'] = meme_data['header'].apply(lambda x: clean_text(x))
meme_data['meaning'] = meme_data['meaning'].apply(lambda x: clean_text(x))
meme_data['meaning'] = meme_data[['header','meaning']].apply(lambda x: add_header(x), axis=1)

In [15]:
#преобразуем дату в датасет hugginface для дальнейшего поиска faiss

meme_dataset = Dataset.from_pandas(meme_data[['link', 'meaning']])
meme_dataset

Dataset({
    features: ['link', 'meaning', '__index_level_0__'],
    num_rows: 2243
})

## 2. Используем готовые модели без дополнительного дообучения

#### Выбор модели.

*Основные моменты*:
- Я решаю задачу симметричного семантического поиска (длина запроса соразмерна длине ответа (текстового описания мема)).
- Модель должна подходить для русского языка.
- Для поиска нужны эмбеддинги предложений, в некоторых моделях такой output отсутсвует, может потребоваться pooling для усреднения эмбеддингов слов.
- Некоторые модели ожидают специальные токены на входных предложениях (например query/passage)
- Существуют Cased (регистр слов не имеет значения) и uncased (слова должны быть в нижнем регистре) модели.

#### Важный момент - оценка качества модели.
- Мой датасет небольшой (около 2000 образцов), и там нет разметки (например, коэффициента, показывающего, насколько конкретный мем подходит к определенному запросу пользователя). Такая разметка планируется в будущем, для этого я буду собирать реакции реальных пользователей на предложенные мемы в моем телеграм-боте.
- Оценить, насколько хорошо отработала модель, я смогла только полагаясь на свои субъективные ощущения.
- Я считала модель подходящей, если она подбирает более-менее релевантные к запросу мемы. 
- Я составила список примеров пользовательских запросов, на которых сравнивала разные модели.

**Вот некоторые примеры, на которых я проводила сравнение:**
- Моя реакция, когда забыл выключить утюг перед тем, как уйти из дома
- Когда ты готовился к экзамену всю ночь, а на следующий день узнаешь, что экзамен перенесли
- Иду на свидание чтобы покушать
- Мое лицо, когда друзья рассказывают про свои приключения на отдыхе, а я не ездил никуда.

#### Модели, которые я использовала (в качестве примера оставила код для первых двух, для остальных процесс похож):
- [sentence-transformers/distiluse-base-multilingual-cased-v1](https://huggingface.co/sentence-transformers/distiluse-base-multilingual-cased-v1)
- [intfloat/multilingual-e5-large](https://huggingface.co/intfloat/multilingual-e5-large) (оказалась наиболее подходящей, далее я также использовала версии base и small)
- [DeepPavlov/rubert-base-cased](https://huggingface.co/DeepPavlov/rubert-base-cased)
- [DeepPavlov/rubert-base-cased-conversational](https://huggingface.co/DeepPavlov/rubert-base-cased-conversational)


#### Model 1 sentence-transformers/distiluse-base-multilingual-cased-v1

In [ ]:
from transformers import AutoTokenizer, TFAutoModel
import warnings
warnings.filterwarnings("ignore")

model_ckpt = "sentence-transformers/distiluse-base-multilingual-cased-v1" 
tokenizer_distiluse = AutoTokenizer.from_pretrained(model_ckpt)
model_distiluse = TFAutoModel.from_pretrained(model_ckpt, from_pt=True)


### Подготовка эмбеддингов

In [61]:
# по умолчанию модель будет векторизировать каждое слово, поэтому нам надо усреднить значения и получить эмбединги предложений
# будем использовать метод CLS pooling на outputs нашей модели.  
# собираем last hidden state где содержится [CLS] token

def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

# функция, которая токенизирует данные и возвращает эмбеддинг предложения

def get_embeddings(text_list):
    encoded_input = tokenizer_distiluse(
        text_list, padding=True, truncation=True, return_tensors="tf"
    )
    encoded_input = {k: v for k, v in encoded_input.items()}
    model_output = model_distiluse(**encoded_input)
    return cls_pooling(model_output)


In [16]:
#протестируем на одном образце
embedding = get_embeddings(meme_dataset["meaning"][0])
embedding.shape

TensorShape([1, 768])

In [ ]:
# получаем эмбеддинги предложений для всего датасета
# переводим значения в numpy формат, чтобы затем передать в faiss 

embeddings_dataset = meme_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["meaning"]).numpy()[0]}
)


In [ ]:
# добавляем к датасету faiss index (особая структура данных которая позволяет выполнять поиск эмбедингов наиболее похожих на эмбединг нашего запроса)
embeddings_dataset.add_faiss_index(column="embeddings")


#### Поиск с помощью FAISS 

In [23]:
question = "мы"

# чистим текст запроса. Важный момент - на этом этапе я еще экспериментировала с чисткой данных, далее от нее отказалась.
question = clean_text(question)

# создаем эмбединг нашего запроса
question_embedding = get_embeddings([question]).numpy()
question_embedding.shape

# теперь будем выполнять поиск ближайших соседей к этому эмбедингу с помощью ф-ии get_nearest_examples()
# функция возвращает кортеж:
# (коэффициент сходства запроса и найденных эмбеддингов, топ-5 наиболее подходящих значений)


scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=5
)

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)

for _, row in samples_df.iterrows():
    print(f"MEME: {row.meaning}")
    print(f"SCORE: {row.scores}")
    print(f"LINK: {row.link}")
    print("=" * 50)
    print()

MEME: озадаченный широкий реакция странный непонятный
SCORE: 8.46925163269043
LINK: https://memepedia.ru/ozadachennyj-kot-za-stolom/

MEME: умирающий тони старок условно разделять тип
SCORE: 8.136177062988281
LINK: https://memepedia.ru/umirayushhij-toni-stark/

MEME: несмотря оба танос идти друг друг становиться отдельный
SCORE: 7.759411811828613
LINK: https://memepedia.ru/realnost-polna-razocharovanij/

MEME: понимать насмешка человек сначала говорить противоречить свой
SCORE: 7.482399940490723
LINK: https://memepedia.ru/vy-ne-ponimaete-eto-drugoe/

MEME: текст звучать следующий
SCORE: 6.23533821105957
LINK: https://memepedia.ru/govoryu-muzhu/



#### Model 2 https://huggingface.co/intfloat/multilingual-e5-large

##### Здесь процесс не отличается от предыдущей модели за исключением некоторых моментов:
- Модель ожидает токен 'query:' в предложениях запросов пользователя и токен 'passage:' в предложениях ответов (текстовые описания мемов).
- В описании к модели сказано, что для симметричного поиска и запрос, и ответ должны содержать токен 'query:'. Я попробовала разные варианты и пришла к выводу, что лучше придерживаться схемы из предыдущего пункта
- В выходных слоях модели уже есть слой sentence_embedding, поэтому пулинг для усреднения не нужен

In [ ]:
# импорты
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F
from torch import Tensor


In [37]:
# загрузка модели и токенайзера
from sentence_transformers import SentenceTransformer
model_e5_large = SentenceTransformer("intfloat/multilingual-e5-large")
tokenizer_e5_large = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')

In [38]:
# ф-я получения эмбеддингов
def get_embedding_e5_large_original(input_texts):
    tokenized_input = tokenizer_e5_large(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')
    model_outputs = model_e5_large(tokenized_input)
    sentence_embedding = model_outputs.sentence_embedding.detach().numpy()
    return sentence_embedding
    
#example
input_texts = all_data["passage"][0]
get_embedding_e5_large_original(input_texts).shape


(1, 1024)

In [311]:
# добавим префикс query: согласно требованиям модели

# Use "query: " and "passage: " correspondingly for asymmetric tasks such as passage retrieval in open QA, ad-hoc information retrieval.
# Use "query: " prefix for symmetric tasks such as semantic similarity, bitext mining, paraphrase retrieval.
# Use "query: " prefix if you want to use embeddings as features, such as linear probing classification, clustering.


def add_prefix(text):
    text = 'passage: '+text
    return text

meme_data["passage"] = meme_data['meaning'].apply(lambda x: add_prefix(x))

In [39]:
# преобразуем дату в датасет hugginface

meme_dataset_e5_large = Dataset.from_pandas(all_data[['link', 'passage']])
meme_dataset_e5_large

Dataset({
    features: ['link', 'passage'],
    num_rows: 2153
})

In [52]:
# получаем эмбеддинги датасета и добавляем к нашему датасету faiss index 
embeddings_dataset_e5_large2 = meme_dataset_e5_large2.map(
    lambda x: {"embeddings": get_embedding_e5_large(x["passage"])[0]}
)

embeddings_dataset_e5_large.add_faiss_index(column="embeddings")

Map:   0%|          | 0/2243 [00:00<?, ? examples/s]

In [54]:
# процесс поиска
question = 'query: '+all_data['example'][4]

#создаем эмбединг нашего запроса
question_embedding = get_embedding_e5_large_original([question])
question_embedding.shape

scores, samples = embeddings_dataset_e5_large.get_nearest_examples(
    "embeddings", question_embedding, k=5
)

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=True, inplace=True)

for _, row in samples_df.iterrows():
    print(f"MEME: {row.passage}")
    print(f"SCORE: {row.scores}")
    print(f"LINK: {row.link}")
    print("=" * 50)
    print()

MEME: passage: Мем по своей сути похож на мем со Светящимся мозгом, который также призван в
несерьезной форме демонстрировать интеллектуальное превосходство.
SCORE: 0.3387693762779236
LINK: https://memepedia.ru/whomst/

MEME: passage: Мем стали искажать, подставляя вместо “поделать” другие глаголы. Так, теперь модно стало говорить (по крайней мере, пока в онлайне) “ля шо бы поделат”, ля шо бы поест”, “ля шо бы послушат”. Никакого дополнительного смысла этот мем не несет.
SCORE: 0.35138118267059326
LINK: https://memepedia.ru/lya-sho-by-podelat/

MEME: passage: Мем с озадаченным котом в широком смысле можно использовать как реакцию на что-то странное, непонятное.
SCORE: 0.35262662172317505
LINK: https://memepedia.ru/ozadachennyj-kot-za-stolom/

MEME: passage: Мем способен высмеять любые проявления ненормального поведения, от которого люди буквально срываются на крик. Всего лишь два кадра с этой сцены могут затронуть серьезные социальные темы или же известные исторические ситуации. Достат

## 3. Fine-tuning моделей

1. На данном этапе я уже дополнила свой датасет примерами пользовательских запросов к каждому из мемов (столбец example в датафрейме) с помощью chatgpt.
2. Буду дообучать модель *multilingual-e5-large* на этом датасете.
3. Loss для обучения. Поскольку в датасете нет score, который бы показывал сходство между мемом и запросом пользователя, то наиболее подходящий loss - *MultipleNegativesRankingLoss*.

- Формирование пар: В процессе обучения для каждого положительного примера (например, текста запроса) алгоритм автоматически выбирает отрицательные примеры из других примеров в том же батче. Таким образом, все остальные примеры в батче, кроме соответствующего положительного, рассматриваются как отрицательные.
- Расчет потерь: Для каждой пары положительного и отрицательного примеров модель вычисляет сходство (обычно это косинусное сходство векторов, представляющих эти примеры). Целью является максимизация сходства между положительными парами и минимизация сходства между положительным и отрицательными примерами.
- Обучение модели: В процессе обучения модель стремится расположить векторы так, чтобы векторы положительных примеров были ближе друг к другу в векторном пространстве, а векторы отрицательных примеров – как можно дальше. Это достигается путем минимизации функции потерь.
- Применение softmax и логарифмирование: После вычисления сходства для всех пар в батче, применяется функция softmax для преобразования этих сходств в вероятностное распределение. Затем, вычисляется логарифм вероятности правильной пары (положительного примера). Функция потерь стремится минимизировать отрицательный логарифм этой вероятности.

Ссылка на ресурс https://ljvmiranda921.github.io/notebook/2017/08/13/softmax-and-the-negative-log-likelihood/


#### 1. Загрузка датасета

In [41]:
all_data = pd.read_csv('all_data.csv', sep='\t', index_col=0)
all_data

,page,header,link,meaning,example,passage,query
0,1,А мне X дороже Y,https://memepedia.ru/a-mne-x-dorozhe-y/,Мем “А мне X дороже Y” (или “А мне X важнее Y”...,Ты обеспокоен экологией? А мне смешные кошки в...,passage: Мем “А мне X дороже Y” (или “А мне X ...,query: Ты обеспокоен экологией? А мне смешные ...
1,1,Рыба с большой головой смотрит на дайвера,https://memepedia.ru/ryba-s-bolshoj-golovoj-sm...,"В мемах, где рыба с большой головой смотрит на...","Твой младший братик наблюдает за тобой, пока т...","passage: В мемах, где рыба с большой головой с...","query: Твой младший братик наблюдает за тобой,..."
2,1,Две девушки в черных платьях,https://memepedia.ru/dve-devushki-v-chernyx-pl...,Мем “Две девушки в черных платьях” показывает ...,На собеседовании два кандидата: одна пришла в ...,passage: Мем “Две девушки в черных платьях” по...,query: На собеседовании два кандидата: одна пр...
3,1,Пивной ежик,https://memepedia.ru/pivnoj-ezhik/,"Мем “Пивной ежик” – это забавный тренд, которы...","Ты просыпаешься по утрам, открываешь новостной...",passage: Мем “Пивной ежик” – это забавный трен...,"query: Ты просыпаешься по утрам, открываешь но..."
4,1,Война древних русов с ящерами,https://memepedia.ru/vojna-drevnix-rusov-s-yas...,Война древних русов с ящерами или битва русов ...,На семейном ужине старший родственник рассказы...,passage: Война древних русов с ящерами или бит...,query: На семейном ужине старший родственник р...
...,...,...,...,...,...,...,...
2148,133,Мальчик у доски,https://memepedia.ru/malchik-u-doski/,Картинку с мальчиком у доски и словом “Тут” из...,"Тот момент, когда ты понимаешь, что любовь к п...",passage: Картинку с мальчиком у доски и словом...,"query: Тот момент, когда ты понимаешь, что люб..."
2149,134,Не расстраивай Леонида Аркадьевича,https://memepedia.ru/ne-rasstraivaj-leonida-ar...,Мем “Не расстраивай Леонида Аркадьевича” испол...,Когда кто-то опять не выключил свет в ванной. ...,passage: Мем “Не расстраивай Леонида Аркадьеви...,query: Когда кто-то опять не выключил свет в в...
2150,134,Девочка с хвостиками,https://memepedia.ru/mem-devochka-s-xvostikami/,Мем с девочкой с хвостиками используется для в...,"Когда ты пытаешься объяснить, почему опять опо...",passage: Мем с девочкой с хвостиками используе...,"query: Когда ты пытаешься объяснить, почему оп..."
2151,134,Танец Медведева,https://memepedia.ru/tanec-medvedeva/,"Шутки про танец Медведева обычно используют, ч...",Когда ты на вечеринке и DJ включает твою любим...,passage: Шутки про танец Медведева обычно испо...,query: Когда ты на вечеринке и DJ включает тво...


### Варианты обучения модели

#### 1. Можно дообучать модель с помощью одной из 2х библиотек - sentence-transformer (более простой) или transformer (более гибкий, возможна доп. настройка)
- sentence-transformer - загружаем модель через model = SentenceTransformer('intfloat/multilingual-e5-large'). Токенайзер подгружается автоматически
- transformer - загружаем модель через AutoModel + AutoTokenizer  

- важный момент: когда мы загружаем модель через AutoModel нужно дополнительно подгрузить модель или ф-ю для пулинга (усреднения токенов по предложениям). НО для дообучения пулинг не нужен, т.к. важнее чтобы модель училась на уровне токенов. через SentenceTransformer пулинг слои обычно подгружаются сами - эта библиотека подходит для быстрого использования уже готовых моделей для работы с предложениями

#### 2. Датасет оборачиваем в генератор DataLoader в обоих случаях (для MultipleNegativesRankingLoss лучше выбрать NoDublicatesDataloader)
- sentence-transformer - каждый пример оборачиваем в класс InputExample
- transformer - можно сделать кастомный класс Dataset, чтобы уместить туда токенайзер и сделать датасет из пандас датафрейма

#### 3. Используем MultipleNegativesRankingLoss 
- позитивные пары ближе друг к другу в векторном пространстве, а негативные дальше
- мой датасет состоит только из позитивных пар query-passage. MultipleNegativesRankingLoss сам подберет негативные 
- batch_size может влиять на кач-во обучения, т.к. больший пакет подберет больше негативных примеров
- если примеры очень похожи, то модель может иметь трудности с тем, чтобы различать примеры

#### 4. Валидация модели
- в моем датасете примерно 2000 примеров. более того, он будет в дальнейшем использоваться для поиска faiss. каждый из них ценен, поэтому делить на train-val довольно трудно
- 1. ВАРИАНТ - вообще не делить, но кач-во можно будет проверить только при поиске FAISS (не очень удобно, нужно ждать пока модель обучится и непонятно, сколько ее обучать, не переобучилась ли она)
 
Для следующих вариантов нужна разметка (например, 0-предложения не схожи, 1-предложения схожи))
- 2. ВАРИАНТ - разделить в соотношении 90-10. НО все равно какие-то примеры не попадут в выборку
- 3. ВАРИАНТ - кросс-валидация - делить датасет на 5 или 10 частей и обучаться на каждой из 5 выборок, затем усреднить оценку


#### 5. Метрики
- пока что у меня нет явных labels, поэтому в качестве основной метрики можно использовать только loss в процессе обучения
- как вариант использовать EmbeddingSimilarityEvaluator - метрика, которая будет оценивать, насколько точно модель определяет сходство между примерами (нужна разметка)
- можно сравнивать среднее расстояние между эмбеддингами позитивных пар и/или асстояние между эмбеддингами позитивных и случайно выбранных негативных пар (нужен валидационный датасет)

#### 6. Нужно ли чистить данные? 
- Трансформеры устроены таким образом, что чистить не обязательно, они могут улавливать определенные закономерности текста. 
- Однако ссылки и какой-то технический шум лучше удалить

#### 7. Какой batch_size использовать? 
- Лучше начать со стандартного 16 или 32. 
- чем больше, тем быстрее обучение, но нужно больше эпох
- но слишком большие пакеты могут ухудшить обучение

#### 8. Сколько эпох? 
- Я выбрала вариант обучать по 3 эпохи, затем смотреть на результаты поиска и обучать дальше

#### 9. Что дальше?
- улучшать FAISS - с помощью настройки
- разметить данные искусственно - добавить негативные примеры и оценки
- можно попробовать Cross-encoder models для искусственной разметки датасета (однако для этого часть датасета должна быть уже размечена, поэтому нужен следующий пункт:)
- система обратной связи - ответ пользователя, является ли мем релевантным или нет (таким образом можно будет разметить часть датасета с помощью реальных пользователей и их запросов)
- фильтрация и ранжирование: Помимо основного поиска по эмбеддингам, можно добавить дополнительные фильтры или ранжирование на основе дополнительных критериев, таких как популярность мема, дата создания и т.д.

Метрики, которые могут быть в дальнейшем использованы, если появится больше данных для валидационного датасета:

In [ ]:
#Cреднее расстояние между эмбеддингами позитивных пар:
#Если это расстояние маленькое, это может указывать на то, что модель правильно "понимает" схожесть позитивных пар.

total_distance = 0
for example in val_examples:
    query_embedding = model.encode(example.texts[0])
    passage_embedding = model.encode(example.texts[1])
    distance = np.linalg.norm(query_embedding - passage_embedding)
    total_distance += distance
average_distance = total_distance / len(val_examples)
print("Average distance for positive pairs:", average_distance)


In [ ]:
#Расстояние между эмбеддингами позитивных и случайно выбранных негативных пар:
#Если это расстояние значительно больше, чем для позитивных пар, это хороший знак. Это означает, что модель может различать позитивные и негативные примеры.

total_distance = 0
num_neg_samples = 0
for example in val_examples:
    query_embedding = model.encode(example.texts[0])
    random_passage = df.sample().iloc[0]['passage']  # выбор случайного passage из всего датасета
    random_passage_embedding = model.encode(random_passage)
    distance = np.linalg.norm(query_embedding - random_passage_embedding)
    total_distance += distance
    num_neg_samples += 1
average_distance = total_distance / num_neg_samples
print("Average distance for random negative pairs:", average_distance)


#### 2. Обучение модели

In [7]:
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

# Создание обучающего датасета
train_examples = [InputExample(texts=[row['query'], row['passage']]) for _, row in all_data[['query', 'passage']].iterrows()]
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)


In [ ]:
# Загрузка модели 
model = SentenceTransformer('intfloat/multilingual-e5-large')

# Определение функции потерь
train_loss = losses.MultipleNegativesRankingLoss(model)

# Обучение модели
num_epochs = 3
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1)

model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
          warmup_steps=warmup_steps, 
          show_progress_bar=True)

os.makedirs('search', exist_ok=True)
model.save('search/search-model2')

#### 3. Добавляем faiss индекс в новый датасет

In [9]:
#создадим ф-ю получения эмбеддингов для новой модели

tokenizer_e5_large = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')

def get_embedding_e5_large(input_texts):
    tokenized_input = tokenizer_e5_large(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')
    model_outputs = model(tokenized_input)
    sentence_embedding = model_outputs.sentence_embedding.detach().numpy()
    return sentence_embedding
    
#example
input_texts = all_data["meaning"][0]
get_embedding_e5_large(input_texts).shape


(1, 1024)

In [ ]:
meme_dataset_faiss = Dataset.from_pandas(all_data[['link', 'query', 'passage']])
meme_dataset_faiss

In [14]:
meme_dataset_faiss = meme_dataset_faiss.map(
    lambda x: {"embeddings": get_embedding_e5_large(x["passage"])[0]}
)


Map:   0%|          | 0/2153 [00:00<?, ? examples/s]

In [ ]:
#добавляем faiss index
meme_dataset_faiss.add_faiss_index(column="embeddings")


In [81]:
question = 'query: '+'Когда решил попробовать новую прическу, а друзья сравнивают тебя с известной личностью, но не в лучшем ее виде'

#создаем эмбединг запроса
question_embedding = get_embedding_e5_large([question])
question_embedding.shape

# поиск
scores, samples = meme_dataset_faiss.get_nearest_examples(
    "embeddings", question_embedding, k=5
)

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=True, inplace=True)

for _, row in samples_df.iterrows():
    print(f"MEME: {row.passage}")
    print(f"SCORE: {row.scores}")
    print(f"LINK: {row.link}")
    print("=" * 50)
    print()

MEME: passage: Тест “На кого из знаменитостей вы похожи?” появился на корейском развлекательном сайте Vonvon 17 марта 2017 года. Приложение быстро стало вирусным среди западных пользователей интернета. Но не из-за его реалистичности, а потому, что оно выдавало притянутые результаты. Так, если загрузить фото чернокожего мужчины, собаки или женщины из Китая, приложение все равно покажет, что вы похожи на Леонардо ДиКаприо или Марго Робби. Похоже, в его базе не так много селебрити, и поэтому все видят одни и те же итоговые картинки.
SCORE: 1.0769195556640625
LINK: https://memepedia.ru/na-kogo-iz-znamenitostej-vy-poxozhi/

MEME: passage: Летом 2018 года появилась фотожаба с лицом Илона Маска, прифотошопленным на фото рэпера Post Malone. Юмор в том, что с заменой лица мало что изменилось.
SCORE: 1.1951813697814941
LINK: https://memepedia.ru/fotozhaby-s-licom-ilona-maska/

MEME: passage: Мем с Рикардой построен на сходстве девушки с Рикардо Милосом. Kalinka Fox не просто скопировала костюм, 

#### 4. Продолжение обучения модели (далее шаги повторяются)

In [66]:
# Загрузка модели 
model2 = SentenceTransformer('search/search-model2')

# Определение функции потерь
train_loss = losses.MultipleNegativesRankingLoss(model2)

# Обучение модели
num_epochs = 3

model2.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs, 
          show_progress_bar=True)



Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/135 [00:00<?, ?it/s]

Iteration:   0%|          | 0/135 [00:00<?, ?it/s]

Iteration:   0%|          | 0/135 [00:00<?, ?it/s]

In [67]:
model2.save('search/search-model2-1')

In [ ]:
# новая ф-я получения эмбеддингов
def get_embedding_e5_large_trained2(input_texts):
    tokenized_input = tokenizer_e5_large(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')
    model_outputs = model2(tokenized_input)
    sentence_embedding = model_outputs.sentence_embedding.detach().numpy()
    return sentence_embedding

meme_dataset_trained2 = Dataset.from_pandas(all_data[['link', 'passage']])

meme_dataset_trained2 = meme_dataset_trained2.map(
    lambda x: {"embeddings": get_embedding_e5_large_trained2(x["passage"])[0]}
)

#добавляем к нашему датасету faiss index (особая структура данных которая позволяет выполнять поиск эмбедингов 
#наиболее похожих на эмбединг нашего запроса)
meme_dataset_trained2.add_faiss_index(column="embeddings")


In [80]:
#поиск

question = 'query: '+'Когда решил попробовать новую прическу, а друзья сравнивают тебя с известной личностью, но не в лучшем ее виде'

#создаем эмбединг нашего запроса
question_embedding = get_embedding_e5_large_trained2([question])
question_embedding.shape

#теперь будем выполнять поиск ближайших соседей к этому эмбедингу с помощью ф-ии get_nearest_examples()
# The Dataset.get_nearest_examples() возвращает кортеж:
#(коэффициент сходства запроса и эмбедингов, топ-5 наиболее подходящих значений)


scores, samples = meme_dataset_trained2.get_nearest_examples(
    "embeddings", question_embedding, k=5
)

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=True, inplace=True)

for _, row in samples_df.iterrows():
    print(f"MEME: {row.passage}")
    print(f"SCORE: {row.scores}")
    print(f"LINK: {row.link}")
    print("=" * 50)
    print()

MEME: passage: Тест “На кого из знаменитостей вы похожи?” появился на корейском развлекательном сайте Vonvon 17 марта 2017 года. Приложение быстро стало вирусным среди западных пользователей интернета. Но не из-за его реалистичности, а потому, что оно выдавало притянутые результаты. Так, если загрузить фото чернокожего мужчины, собаки или женщины из Китая, приложение все равно покажет, что вы похожи на Леонардо ДиКаприо или Марго Робби. Похоже, в его базе не так много селебрити, и поэтому все видят одни и те же итоговые картинки.
SCORE: 1.0796183347702026
LINK: https://memepedia.ru/na-kogo-iz-znamenitostej-vy-poxozhi/

MEME: passage: Мем про “барби шоп” высмеивает нелепые прически парней. Нередко там появляются совсем пародийные картинки. Сам текст стал вирусной пастой, его пишут обычные пользователи, иронизируя над тем, как их плохо постригли.
SCORE: 1.1855273246765137
LINK: https://memepedia.ru/sxodil-v-etot-vash-barbi-shop/

MEME: passage: Флешмоб с переделыванием лиц Эльзы и других

In [ ]:
# дальнейшее обучение модели
model3 = SentenceTransformer('search/search-model2-1')

# Определение функции потерь
train_loss = losses.MultipleNegativesRankingLoss(model3)

num_epochs = 3

model3.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs, 
          show_progress_bar=True)

model3.save('search/search-model2-2')

In [ ]:
# ф-я получения эмбеддингов с новой версией модели
import torch.nn.functional as F

model3 = SentenceTransformer('search/search-model2-2')
tokenizer_e5_large = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')

def get_embedding_e5_large_trained3(input_texts):
    tokenized_input = tokenizer_e5_large(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')
    model_outputs = model3(tokenized_input)
    sentence_embedding = model_outputs.sentence_embedding
    
    #нормализуем эмбеддинг
    sentence_embedding = F.normalize(sentence_embedding, p=2, dim=1)
    return sentence_embedding.detach().numpy()

meme_dataset_trained3 = Dataset.from_pandas(all_data[['link', 'passage']])

meme_dataset_trained3 = meme_dataset_trained3.map(
    lambda x: {"embeddings": get_embedding_e5_large_trained3(x["passage"])[0]}
)

#добавляем faiss index
meme_dataset_trained3.add_faiss_index(column="embeddings")


In [25]:
# поиск
question = 'query: '+'Когда ты готовился к экзамену всю ночь, а на следующий день узнаешь, что экзамен перенесли'

#создаем эмбединг нашего запроса
question_embedding = get_embedding_e5_large_trained3([question])
question_embedding.shape

scores, samples = meme_dataset_trained3.get_nearest_examples(
    "embeddings", question_embedding, k=5
)

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=True, inplace=True)

for _, row in samples_df.iterrows():
    print(f"MEME: {row.passage}")
    print(f"SCORE: {row.scores}")
    print(f"LINK: {row.link}")
    print("=" * 50)
    print()

MEME: passage: Мем “Крош орет” или “Крош кричит” иллюстрирует ситуации, когда человеку так плохо, что ему ничего не остается кроме того, как кричать. Это могут быть и гиперболизированные ситуации, когда ты узнаешь о дополнительном уроке. Аналогичный мем из прошлого – с кричащим ковбоем из клипа Big Enough Джимми Барнас.
SCORE: 1.2425875663757324
LINK: https://memepedia.ru/krosh-krichit/

MEME: passage: Мемы с плачущим Макконахи – реакция человека на что-то упущенное, обидное. Часто их используют в ироническом ключе, “заставляя” Купера плакать, например, из-за трейлера новых “Звездных войн”.
SCORE: 1.2528679370880127
LINK: https://memepedia.ru/makkonaxi-plachet/

MEME: passage: Мем “Давайте после майских” – это ироничная отмазка от любой, самой незначительной задачи. В интернете принято высмеивать тех, кто постоянно откладывает дела из-за приближающихся праздников. Причем начинают это делать обычно задолго до самих выходных.
SCORE: 1.2533793449401855
LINK: https://memepedia.ru/davajte-p

### 4. Дополнительная настройка FAISS 

Я попробовала 2 вида индексов FAISS и не заметила никакой разницы между результатами:
- faiss.IndexFlatIP - Косинусное сходство измеряет угол между двумя векторами, не учитывая их длину. После L2 нормализации все вектора имеют единичную длину, что делает их сравнение на основе косинусного сходства более эффективным. В этом случае, расстояние между векторами определяется исключительно углом между ними.
- faiss.IndexFlatL2 - Евклидово расстояние измеряет прямое расстояние между двумя точками в пространстве. Когда применяется L2 нормализация, уменьшается влияние различий в длине векторов, делая расстояния более схожими с углами, измеряемыми при косинусном сходстве.
- Такой результат получился т.к., после l2 нормализации обе метрики становятся идентичными, т.к. они учитывают только направление векторов, а не их длину



In [11]:
#загрузка модели и токенайзера
from sentence_transformers import SentenceTransformer, InputExample, losses, models, datasets
import torch.nn.functional as F

model3 = SentenceTransformer('search/search-model2-2')
tokenizer_e5_large = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')

def get_embedding_e5_large_trained3(input_texts):
    tokenized_input = tokenizer_e5_large(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')
    model_outputs = model3(tokenized_input)
    sentence_embedding = model_outputs.sentence_embedding
    #нормализуем эмбеддинг
    sentence_embedding = F.normalize(sentence_embedding, p=2, dim=1) 
    return sentence_embedding.detach().numpy()

In [13]:
# подготовка датасета и получение эмбеддингов
import faiss
from datasets import Dataset

meme_dataset_trained3 = Dataset.from_pandas(all_data[['link', 'passage']])

meme_dataset_trained3 = meme_dataset_trained3.map(
    lambda x: {"embeddings": get_embedding_e5_large_trained3(x["passage"])[0]}
)
embeddings = np.vstack(meme_dataset_trained3['embeddings'])

Map:   0%|          | 0/2153 [00:00<?, ? examples/s]

In [14]:
# определяем индекс faiss
index = faiss.IndexFlatIP(1024) #мера схожести - косинусное расстояние
index.add(embeddings)

In [34]:
# Ищем ближайших соседей в FAISS индексе на основании IndexFlatIP (косинусное сходство)
question = 'query: '+'иду на свидание чтобы покушать'

#создаем эмбединг нашего запроса
question_embedding = get_embedding_e5_large_trained3([question])

k = 5  # Количество ближайших соседей
distances, indices = index.search(question_embedding, k)

# Создание нового DataFrame с результатами поиска
result_df = pd.DataFrame({'query_link': [all_data['link'][i] for i in indices[0]],
                           'query_passage': [all_data['passage'][i] for i in indices[0]],
                           'distance': distances[0]})

result_df = result_df.sort_values('distance', ascending=False)

for _, row in result_df.iterrows():
    print(f"MEME: {row.query_passage}")
    print(f"SCORE: {row.distance}")
    print(f"LINK: {row.query_link}")
    print("=" * 50)
    print()

MEME: passage: Тренд “Первое свидание” иронично показывает разницу между парнями и девушками. Тогда как девушки на первое свиданиe одеваются эффектно, мужчины в мемах предпочитают самые безумные и нестандартные наряды. Часто чтобы это показать, используют кадры из игр, фильмов и сериалов.
SCORE: 0.4013262987136841
LINK: https://memepedia.ru/pervoe-svidanie-ona-i-ya/

MEME: passage: Мем с девочкой, которая убегает после поцелуя, используется для высмеивания парней, испытывающих проблемы в общении с противоположным полом. Сам момент выглядит довольно комично, плюс внимание привлекают эмоции на лице разочарованного парнишки.
SCORE: 0.3746700882911682
LINK: https://memepedia.ru/devochka-ubegaet-ot-malchika-posle-poceluya/

MEME: passage: Слово “кусь” – это сокращение от слова “укус”, но значения у него могут быть самыми разными. Прежде всего, “кусь” означает легкий укус как проявление симпатии. Также это слово может означать поцелуй и использоваться вместо приветствия.
SCORE: 0.36613446474

In [16]:
# мера схожести - евклидово расстояние
index2 = faiss.IndexFlatL2(1024) 
index2.add(embeddings)

In [30]:
# Ищем ближайших соседей в FAISS индексе на основании IndexFlatL2 (евклидово расстояние)
question = 'query: '+'иду на свидание чтобы покушать'

#создаем эмбединг нашего запроса
question_embedding = get_embedding_e5_large_trained3([question])

k = 5  # Количество ближайших соседей
distances, indices = index2.search(question_embedding, k)

# Создание нового DataFrame с результатами поиска
result_df = pd.DataFrame({'query_link': [all_data['link'][i] for i in indices[0]],
                           'query_passage': [all_data['passage'][i] for i in indices[0]],
                           'distance': distances[0]})

result_df = result_df.sort_values('distance', ascending=True)

for _, row in result_df.iterrows():
    print(f"MEME: {row.query_passage}")
    print(f"SCORE: {row.distance}")
    print(f"LINK: {row.query_link}")
    print("=" * 50)
    print()

MEME: passage: Тренд “Первое свидание” иронично показывает разницу между парнями и девушками. Тогда как девушки на первое свиданиe одеваются эффектно, мужчины в мемах предпочитают самые безумные и нестандартные наряды. Часто чтобы это показать, используют кадры из игр, фильмов и сериалов.
SCORE: 1.1973477602005005
LINK: https://memepedia.ru/pervoe-svidanie-ona-i-ya/

MEME: passage: Мем с девочкой, которая убегает после поцелуя, используется для высмеивания парней, испытывающих проблемы в общении с противоположным полом. Сам момент выглядит довольно комично, плюс внимание привлекают эмоции на лице разочарованного парнишки.
SCORE: 1.2506600618362427
LINK: https://memepedia.ru/devochka-ubegaet-ot-malchika-posle-poceluya/

MEME: passage: Слово “кусь” – это сокращение от слова “укус”, но значения у него могут быть самыми разными. Прежде всего, “кусь” означает легкий укус как проявление симпатии. Также это слово может означать поцелуй и использоваться вместо приветствия.
SCORE: 1.26773118972

- Далее я решила проверить, используется ли в базовой модели нормализация по умолчанию?
- Вывод: да, в данной модели из библиотеки sentence-transformers она используется, поэтому нет разницы между использованием IndexFlatIP и IndexFlatL2

In [2]:
# проверка нормы векторов
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('intfloat/multilingual-e5-large')
sentences = ["Пример предложения", "Еще одно предложение"]
embeddings = model.encode(sentences)

# Проверка нормы эмбеддингов
norms = np.linalg.norm(embeddings, axis=1)
print(norms)


[0.99999994 1.        ]


### 5. Подготовка к деплою

In [ ]:
# сохранение эмбеддингов обученной модели
embeddings = np.vstack(meme_dataset_trained3['embeddings'])
np.save('embeddings.npy', embeddings)


In [ ]:
# экспорт модели в onnx-формат (для оптимизации размера докер-контейнера)
import torch.onnx

input_texts = 'Привет как дела'
inp = tokenizer(input_texts, max_length=128, padding="max_length", truncation=True, return_tensors='np')
input_ids = inp.input_ids
attention_mask = inp.attention_mask
dummy_input = (input_ids, attention_mask)

torch.onnx.export(model, dummy_input, "small_model3.onnx")

In [ ]:
# пример инференса с помощью onnx-сессии
ort_session = ort.InferenceSession("small_model3.onnx")
text = "query: Когда ты готовился к экзамену всю ночь, а на следующий день узнаешь, что экзамен перенесли"
inputs = tokenizer(text, max_length=128, padding="max_length", truncation=True, return_tensors='np')

ort_inputs = {ort_session.get_inputs()[0].name: inputs['input_ids'],
              ort_session.get_inputs()[1].name: inputs['attention_mask']}

sentence_embeddings = ort_session.run(None, ort_inputs)[0]